In [6]:
"""
@Author: blankxiao
@file: A_2.py
@Created: 2024-09-06 14:03
@Desc: A_2 求出碰撞时间 和当时各个部分的状态(位置和速度)
主要是求碰撞时间 然后代入A_1模型 得出碰撞后各个部分的位置和速度
"""

import pandas as pd
import numpy as np
import sympy as sp

from A_1 import get_alpha_0, get_r_0, get_x_y, get_r_i, get_v_i, get_alpha_i, loong_head_speed


def get_pos_of_corner(alpha_i: int, alpha_i_pre: int, r_i: int, r_i_pre: int, r_0: int, r_1: int, alpha_0: int, alpha_1: int):
    """
    获取三个点的坐标
    """

    # 孔距边界距离 m
    AD_d = 0.275
    # 板凳长度的一半
    CD_d = 0.15
    # A C的距离
    AC_d = sp.sqrt(AD_d ** 2 + CD_d ** 2)

    # 龙头两个把手的间距
    L_loong_head = 3.41 - 2 * AD_d
    # 龙身两个把手的间距
    L_loong_body = 2.20 - 2 * AD_d
    # 龙头两个把手的间距
    D_1 = L_loong_head
    # 龙身两个把手的间距
    D_2 = L_loong_body

    Gama = sp.atan(CD_d / AD_d)

    gama_i_2 = - Gama + alpha_i_pre - alpha_i + sp.asin(r_i / D_2 * sp.sin(alpha_i_pre - alpha_i))
    gama_i_1 = - Gama + alpha_i_pre - alpha_i + sp.asin(r_i_pre / D_2 * sp.sin(alpha_i_pre - alpha_i))
    gama_0 = Gama + alpha_0 - alpha_1 + sp.asin(r_1 / D_1 * sp.sin(alpha_0 - alpha_1))

    r_B_i = sp.sqrt(AC_d ** 2 + r_i ** 2 - 2 * AC_d * r_i * sp.cos(gama_i_1))
    r_B_i_pre = sp.sqrt(AC_d ** 2 + r_i_pre ** 2 - 2 * AC_d * r_i_pre * sp.cos(gama_i_2) )
    r_C_0 = sp.sqrt(AC_d ** 2 + r_0 ** 2 - 2 * AC_d * r_0 * sp.cos(gama_0) )

    beta_B_i = alpha_i_pre + sp.asin(AC_d / r_B_i * sp.sin(gama_i_1))
    beta_B_i_pre = alpha_i - sp.asin(AC_d / r_B_i_pre * sp.sin(gama_i_2))
    beta_C_0 = alpha_0 + sp.asin(AC_d / r_0 * sp.sin(gama_0))

    x_B_i, y_B_i = get_x_y(alpha=beta_B_i, r=r_B_i)
    x_B_i_pre, y_B_i_pre = get_x_y(alpha=beta_B_i_pre, r=r_B_i_pre)
    x_C_0, y_C_0 = get_x_y(alpha=beta_C_0, r=r_C_0)

    return (x_B_i, y_B_i), (x_B_i_pre, y_B_i_pre), (x_C_0, y_C_0)

def get_end_time(a: int):
    """
    获取龙头到达原点的时间 并将位置信息存入df_rav
    """
    t = sp.var("t")

    # 将 get_alpha_0 代入 get_r_0，得到关于 t 的函数
    alpha_0 = get_alpha_0(t, a)
    r_0 = get_r_0(alpha_0, a)
    # 求解方程 r_0 = 0
    equation = sp.Eq(r_0, 0)
    return float(sp.solve(equation, t)[0])


def forward_vaild(r_0: int, r_1: int, spiral_d: int):
    """
    @param r_0: 龙头极径
    @param r_1: 第一节龙身极径
    返回是否是锐角
    """
    theta_0 = 2 * np.pi * r_0 / spiral_d
    theta_1 = 2 * np.pi * r_1 / spiral_d

    detal_x = (r_0 * sp.cos(theta_0) - r_1 * sp.cos(theta_1))
    dx = sp.cos(theta_0 - theta_0 * sp.sin(theta_0))
    detal_y = r_0 * sp.sin(theta_0) - r_1 * sp.sin(theta_1)
    dy = sp.sin(theta_0 + theta_0 * sp.cos(theta_0))
    return  detal_x * dx + detal_y * dy >= 0


def are_collinear(A, B, C, atol=1e-1):
    """
    判断三个极坐标点 A、B、C 是否共线
    :param A: 点 A 的直角坐标
    :param B: 点 B 的直角坐标
    :param C: 点 C 的直角坐标
    :return: 如果 A、B、C 共线，返回 True
    """
    # 将极坐标转换为笛卡尔坐标
    x_A, y_A = [A[0], A[1]]
    x_B, y_B = [B[0], B[1]]
    x_C, y_C = (C[0], C[1])
    
    # 计算向量 AB 和 AC
    vector_AB = (x_B - x_A, y_B - y_A)
    vector_AC = (x_C - x_A, y_C - y_A)
    
    # 计算向量 AB 和 AC 的叉积
    cross_product = vector_AB[0] * vector_AC[1] - vector_AB[1] * vector_AC[0]
    # 共线 且 A在BC之间
    if abs(cross_product) < 0.05 and (x_A - x_B) * (x_A - x_C) <= 2:
        print(cross_product, (x_A - x_B) * (x_A - x_C))
    return np.isclose(cross_product, 0, atol=atol) and np.isclose((x_A - x_B) * (x_A - x_C), 0, atol=atol * 5)




In [9]:


def get_crash_r_0(a: int, num_point=15, times=0):
    """
    @param a: 螺线距离
    @param num_point: 轨迹点数
    @return: r0 碰撞时的r0
    """
    num_point += (9 - times) * 5
    end = get_end_time(a)
    print(f"{a=}到达的零点时间 {end=}")
    # 认为碰撞仅存在2-10 这些点
    time_range = np.arange(end - 100, end - 10, 0.05).tolist()

    df_rav = pd.DataFrame(index=pd.MultiIndex.from_product([range(num_point), ['r', 'alpha', "v"]]), columns=time_range)

    # df_xyv = pd.DataFrame(index=pd.MultiIndex.from_product([range(num_point), ['x', 'y', "v"]]), columns=time_range)
    # df_coner = pd.DataFrame(index=pd.MultiIndex.from_product([range(num_point), ['B_i', 'B_i_pre', "C_0"]]), columns=time_range)

    r_0 = 0
    for t in time_range:
        for point_index in range(num_point):
            if point_index != 0:
                r_i_pre=df_rav[t][point_index - 1, 'r']
                v_i_pre=df_rav[t][point_index - 1, 'v']

                cur_r = get_r_i(r_i_pre=r_i_pre, point_index=point_index, spiral_d=a)
                cur_alpha = get_alpha_i(r_i=cur_r, spiral_d=a)

                cur_v = get_v_i(v_i_pre=v_i_pre, r_i_pre=r_i_pre, r_i=cur_r, spiral_d=a)
            else:
                cur_alpha = get_alpha_0(t, spiral_d=a)
                cur_r = get_r_0(cur_alpha, spiral_d=a)
                cur_v = loong_head_speed

            # 记录当前的 r 和 alpha
            df_rav.at[(point_index, 'r'), t] = cur_r
            df_rav.at[(point_index, 'alpha'), t] = cur_alpha
            df_rav.at[(point_index, 'v'), t] = cur_v
            
            if point_index > 2:
                alpha_0 = df_rav.at[(0, 'alpha'), t]
                r_0 = df_rav.at[(0, 'r'), t]

                alpha_1 = df_rav.at[(1, 'alpha'), t]
                r_1 = df_rav.at[(1, 'r'), t]

                alpha_i_pre = df_rav.at[(point_index - 1, 'alpha'), t]
                r_i_pre = df_rav.at[(point_index - 1, 'r'), t]

                B_i, B_i_pre, C_0 = get_pos_of_corner(alpha_0=alpha_0, r_0=r_0, alpha_1=alpha_1, r_1=r_1, alpha_i_pre=alpha_i_pre, r_i_pre=r_i_pre, alpha_i=cur_alpha, r_i=cur_r )
                if are_collinear(B_i, B_i_pre, C_0):
                    print(f'{t}时刻发现碰撞点 r_0为{r_0}')
                    return r_0
    return r_0



In [10]:

step = 0.1
spiral_d = 0.55
times = 10
crash_dic = {}
while times := times - 1:
    crash_r_0 = get_crash_r_0(a=spiral_d, times=times)
    print(f'{spiral_d=}m')
    print(f'{crash_r_0=}m')
    if crash_r_0 > 4.5:
        spiral_d += step
        step /= 2
    else:
        spiral_d -= step
        step /= 2
    print()
    crash_dic[spiral_d] = crash_r_0
print(crash_dic)

a=0.55到达的零点时间 end=442.3362456254434
379.3362456254518时刻发现碰撞点 r_0为3.32105970380943
spiral_d=0.55m
crash_r_0=3.32105970380943m

a=0.45000000000000007到达的零点时间 end=361.9114736935443
-0.0199562378221972 0.401358923681214
261.9114736935443时刻发现碰撞点 r_0为3.78469878303024
spiral_d=0.45000000000000007m
crash_r_0=3.78469878303024m

a=0.4000000000000001到达的零点时间 end=321.69908772759493
-0.0489861431851553 0.949045343534354
-0.0465543709532253 0.939584724384206
-0.0441029211290709 0.929800839631545
-0.0416317715292644 0.919704720716557
-0.0391408999842111 0.909307626151082
-0.0366302843386521 0.898621028067368
-0.0340999024517265 0.887656598586666
-0.0315497321971092 0.876426196024313
-0.0289797514637980 0.864941850945202
-0.0263899381553110 0.853215752080863
-0.0237802701904869 0.841260232123918
-0.0211507255041323 0.829087753413002
-0.0185012820460364 0.816710893520388
-0.0158319177825227 0.804142330761779
-0.0131426106955557 0.791394829633734
-0.0104333387837596 0.778481226200044
-0.00770408006198688 

ValueError: Could not find root within given tolerance. (0.851413777833211502834 > 2.16840434497100886801e-19)
Try another starting point or tweak arguments.

In [4]:
print(crash_dic)


NameError: name 'crash_dic' is not defined

In [10]:

step = 0.1
spiral_d = 0.55
times = 10
while times := times - 1:
    crash_r_0 = get_crash_r_0(a=spiral_d)
    print(f'{spiral_d=}m')
    print(f'{crash_r_0=}m')
    if crash_r_0 > 4.5:
        spiral_d += step
        step /= 2
    else:
        spiral_d -= step
        step /= 2
    print()


a=0.55到达的零点时间 end=442.3362456254434
379.3362456254518时刻发现碰撞点 r_0为3.32105970380943
spiral_d=0.55m
crash_r_0=3.32105970380943m

a=0.45000000000000007到达的零点时间 end=361.9114736935443
-0.0199562378221972 0.401358923681214
261.9114736935443时刻发现碰撞点 r_0为3.78469878303024
spiral_d=0.45000000000000007m
crash_r_0=3.78469878303024m

a=0.4000000000000001到达的零点时间 end=321.69908772759493
-0.176926378075854 0.467124140863009
-0.174868522450060 0.502122042146058
-0.172738626509307 0.537085727502735
-0.170536508476342 0.571867056988561
-0.168261986599669 0.606321264207057
-0.165914879158311 0.640307412506656
-0.163495004466476 0.673688833945420
-0.161002180877908 0.706333549599680
-0.158436226790632 0.738114669860523
-0.155796960651939 0.768910773428272
-0.153084200962711 0.798606263793343
-0.150297766282580 0.827091702067009
-0.147437475234761 0.854264115111111
-0.144503146510715 0.880027278002642
-0.141494598875367 0.904291969955174
-0.138411651172126 0.926976202913346
-0.135254122327399 0.948005422135783


ValueError: Could not find root within given tolerance. (5400.73337286759670954 > 2.16840434497100886801e-19)
Try another starting point or tweak arguments.